To run this model directly in the browser with zero setup, open it in [Colab here](https://colab.research.google.com/github/sararob/keras-wine-model/blob/master/keras-wide-deep.ipynb).

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [3]:
# Install the latest version of TensorFlow
!pip install -q -U tensorflow==1.7.0

In [5]:
import itertools
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
layers = keras.layers

# This code was tested with TensorFlow v1.7
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 1.7.0


In [ ]:
# Get the data: original source is here: https://www.kaggle.com/zynicide/wine-reviews/data
URL = "https://storage.googleapis.com/sara-cloud-ml/wine_data.csv"
path = tf.keras.utils.get_file(URL.split('/')[-1], URL)


In [3]:
# Convert the data to a Pandas data frame
data = pd.read_csv(path)

In [4]:
# Shuffle the data
data = data.sample(frac=1)

# Print the first 5 rows
data.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
90233,90233,France,This organic Chardonnay offers notes of soft p...,O Made From Organic Grapes,85,18.0,Languedoc-Roussillon,Pays d'Oc,NaN,Chardonnay,Gilles Louvet
75984,75984,US,Shows This wine boasts Petite Sirah's full-bod...,NaN,89,32.0,California,Paso Robles,Central Coast,Petite Sirah,Proulx
8459,8459,France,Faiveley owns two acres of this large Grand Cr...,NaN,93,226.0,Burgundy,Echézeaux,NaN,Pinot Noir,Domaine Faiveley
34267,34267,Italy,This oak-driven Aglianico opens with aromas of...,Jumara,86,25.0,Southern Italy,Irpinia,NaN,Aglianico,I Capitani
97811,97811,New Zealand,"A lot has changed at Cloudy Bay, but the wine ...",NaN,89,25.0,Marlborough,NaN,NaN,Sauvignon Blanc,Cloudy Bay


In [5]:
# Do some preprocessing to limit the # of wine varities in the dataset
data = data[pd.notnull(data['country'])]
data = data[pd.notnull(data['price'])]
data = data.drop(data.columns[0], axis=1) 

variety_threshold = 500 # Anything that occurs less than this will be removed.
value_counts = data['variety'].value_counts()
to_remove = value_counts[value_counts <= variety_threshold].index
data.replace(to_remove, np.nan, inplace=True)
data = data[pd.notnull(data['variety'])]

In [6]:
# Split data into train and test
train_size = int(len(data) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

Train size: 95646
Test size: 23912


In [8]:
# Train features
description_train = data['description'][:train_size]
variety_train = data['variety'][:train_size]

# Train labels
labels_train = data['price'][:train_size]

# Test features
description_test = data['description'][train_size:]
variety_test = data['variety'][train_size:]

# Test labels
labels_test = data['price'][train_size:]

In [9]:
# Create a tokenizer to preprocess our text descriptions
vocab_size = 12000 # This is a hyperparameter, experiment with different values for your dataset
tokenize = keras.preprocessing.text.Tokenizer(num_words=vocab_size, char_level=False)
tokenize.fit_on_texts(description_train) # only fit on train

In [10]:
# Wide feature 1: sparse bag of words (bow) vocab_size vector 
description_bow_train = tokenize.texts_to_matrix(description_train)
description_bow_test = tokenize.texts_to_matrix(description_test)

In [11]:
# Wide feature 2: one-hot vector of variety categories

# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(variety_train)
variety_train = encoder.transform(variety_train)
variety_test = encoder.transform(variety_test)
num_classes = np.max(variety_train) + 1

# Convert labels to one hot
variety_train = keras.utils.to_categorical(variety_train, num_classes)
variety_test = keras.utils.to_categorical(variety_test, num_classes)

In [12]:
# Define our wide model with the functional API
bow_inputs = layers.Input(shape=(vocab_size,))
variety_inputs = layers.Input(shape=(num_classes,))
merged_layer = layers.concatenate([bow_inputs, variety_inputs])
merged_layer = layers.Dense(256, activation='relu')(merged_layer)
predictions = layers.Dense(1)(merged_layer)
wide_model = keras.Model(inputs=[bow_inputs, variety_inputs], outputs=predictions)

In [13]:
wide_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
print(wide_model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12000)        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 12040)        0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 256)          3082496     concatenate_1[0][0]              
__________

In [14]:
# Deep model feature: word embeddings of wine descriptions
train_embed = tokenize.texts_to_sequences(description_train)
test_embed = tokenize.texts_to_sequences(description_test)

max_seq_length = 170
train_embed = keras.preprocessing.sequence.pad_sequences(
    train_embed, maxlen=max_seq_length, padding="post")
test_embed = keras.preprocessing.sequence.pad_sequences(
    test_embed, maxlen=max_seq_length, padding="post")

In [15]:
# Define our deep model with the Functional API
deep_inputs = layers.Input(shape=(max_seq_length,))
embedding = layers.Embedding(vocab_size, 8, input_length=max_seq_length)(deep_inputs)
embedding = layers.Flatten()(embedding)
embed_out = layers.Dense(1)(embedding)
deep_model = keras.Model(inputs=deep_inputs, outputs=embed_out)
print(deep_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 170)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 170, 8)            96000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1360)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1361      
Total params: 97,361
Trainable params: 97,361
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
deep_model.compile(loss='mse',
                       optimizer='adam',
                       metrics=['accuracy'])

In [17]:
# Combine wide and deep into one model
merged_out = layers.concatenate([wide_model.output, deep_model.output])
merged_out = layers.Dense(1)(merged_out)
combined_model = keras.Model(wide_model.input + [deep_model.input], merged_out)
print(combined_model.summary())

combined_model.compile(loss='mse',
                       optimizer='adam',
                       metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12000)        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 170)          0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 12040)        0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________

In [18]:
# Run training
combined_model.fit([description_bow_train, variety_train] + [train_embed], labels_train, epochs=10, batch_size=128)

Epoch 1/10
95646/95646 [==============================] - 52s 546us/step - loss: 1165.0368 - acc: 0.0276
Epoch 2/10
95646/95646 [==============================] - 57s 593us/step - loss: 915.0286 - acc: 0.0352
Epoch 3/10
95646/95646 [==============================] - 58s 608us/step - loss: 785.9011 - acc: 0.0396
Epoch 4/10
95646/95646 [==============================] - 56s 590us/step - loss: 668.5673 - acc: 0.0445
Epoch 5/10
95646/95646 [==============================] - 55s 576us/step - loss: 550.9641 - acc: 0.0483
Epoch 6/10
95646/95646 [==============================] - 55s 571us/step - loss: 440.3762 - acc: 0.0563
Epoch 7/10
95646/95646 [==============================] - 57s 594us/step - loss: 342.1391 - acc: 0.0651
Epoch 8/10
95646/95646 [==============================] - 59s 615us/step - loss: 260.5558 - acc: 0.0734
Epoch 9/10
95646/95646 [==============================] - 55s 579us/step - loss: 199.9219 - acc: 0.0853
Epoch 10/10
95646/95646 [==============================] - 57s 

In [19]:
combined_model.evaluate([description_bow_test, variety_test] + [test_embed], labels_test, batch_size=128)

23912/23912 [==============================] - 6s 252us/step


[583.1852051067129, 0.06461191035036899]

In [20]:
# Generate predictions
predictions = combined_model.predict([description_bow_test, variety_test] + [test_embed])

In [27]:
# Compare predictions with actual values for the first few items in our test dataset
num_predictions = 40
diff = 0

for i in range(num_predictions):
    val = predictions[i]
    print(description_test.iloc[i])
    print('Predicted: ', val[0], 'Actual: ', labels_test.iloc[i], '\n')
    diff += abs(val[0] - labels_test.iloc[i])

Not a whole lot to say about the soft, flat nose. It's o.k., but that's really about it. The palate, meanwhile, turns sweet, nearly to the point of candied. And the finish is downright sugary. Decent weight and feel help it along.
Predicted:  16.776217 Actual:  14.0 

This 70-30 blend of Chardonnay and Trebbiano is driven by a strong mineral vein that imparts dusty notes of slate, vitamin and granite over fuller aromas of exotic fruit and peach. The wine is simple and tart in the mouth with a creamy, soft feel overall.
Predicted:  19.51367 Actual:  18.0 

Light, fresh aromas of tomato and plum are gritty. This feels round for the most part, with rubbery tannins. Medicinal, slightly minty plum and berry flavors end with acceptable rawness and a latex note.
Predicted:  21.455168 Actual:  10.0 

A densely fruity wine, with berry and plum fruits, pushed by some firm tannins. It is the ripe fruit, brimming over the glass, that makes this wine so delicious and approachable now.
Predicted:  2

In [28]:
# Compare the average difference between actual price and the model's predicted price
print('Average prediction difference: ', diff / num_predictions)

Average prediction difference:  8.719513726234435
